In [1]:
import numpy as np
from numpy import sqrt
from numpy import sin
from numpy import pi
from numpy import cos
from numpy import exp
from numpy import tanh
from numpy import zeros
from numpy import arccos
from numpy import log10
from matplotlib import pyplot as plt
#import scipy.optimize as opt
#import emcee
from pylab import plot
from scipy import integrate
import corner
import random
from collections import Counter
import time
import scipy
from scipy.stats import poisson # use as poisson.pmf(number of events , mean value)
from scipy import optimize

# All the functions

In [2]:
# we work in geometrized units
Msol = 1.47680000000000000000000000000000000 # in km
c = 1 # we have actually put them to one everywhere in the code already
G = 1
year = 60*60*24*(365.250000000000000000000000000); # s
cc = 2.9980000000000000000000000000000*10**5 # velocity of light in Km/s
pc = 3.0856770000000000000000000000*10**13 # km

# 1 / tau accretion, units?
factor = 22*10**(-3)*(3154*10**(-3)*10**7*10**6)**(-1)

# LISA noise parameters
Amp = 5/10*18/10*10**-44
alpha = 138/1000
beta = -221
kappa = 521
gamma = 1680
fk = 113/10**5
L0 = 25/10*10**9
fstar = 1909/100*10**-3
pm = 10**-12

skyav=(4/5*sqrt(2)*sin(pi/3))


def nu(m1,m2):
    return m1*m2/((m1+m2)**2)

def Mc(m1,m2):
    return (m1 + m2) * (m1*m2/((m1+m2)**2))**(3/5)


In [3]:
# noise function, checked with Mathematica

def Sacc(f):
    return (3*10**-15)**2*(1+(4/10*10**-3/f)**2)*(1+(f/(8*10**-3))**4)

def Sgal(f):
    return Amp*f**(-7/3)*exp(-f**alpha+beta*f*sin(kappa*f))*(1+tanh(gamma*(fk-f)))

def Soms(f):
    return (10*pm)**2*(1+(2*10**-3/f)**4)

def Sacc(f):
    return (3*10**-15)**2*(1+(4/10*10**-3/f)**2)*(1+(f/(8*10**-3))**4)
    
def SnSA(f):
    return 10/3*(4*Sacc(f)/(2*pi*f)**4+Soms(f))/(L0**2)*(1+6/10*(f/fstar)**2)

def R(f):
    return 3/20*2

def Sn(f):
    return abs(SnSA(f)+Sgal(f))

def Pn(f): # gravitational wave frequency
    return abs(Sn(f)*R(f))


# noise function from acceleration paper

#def SaccA(f):
 #   return 9. * 10**(-30)/ (2. * pi * f)**4 * (1. + (6. *10**(-4)/f)**2 * (1+(2.22*10**(-4)/f)**8) )

#def Sconf(f):
 #   A = 3. /20. * 3.2665 * 10**(-44)
  #  s1 = 3014.3
  #  a = 1.183
  #  s2 = 2957.7
  #  k = 2.0928 * 10**(-3)
  #  return A/2. * exp(-s1*f**a) * f**(-7/3) * (1. - tanh(s2*(f-k)))

#def SnA(f):
 #   Sother = 8.899 * 10**(-23)
 #   L = 2.5 * 10**9 
 #   return (4.*SaccA(f) + Sother)/L**2 * (1. + (2.*f*L/0.41/cc/10**3)**2) + Sconf(f)


In [4]:
# waveform

def ampl(m1,m2,d): 
    return (Mc(m1,m2)*G)**(5/6)*sqrt(5/24)/(pi**(2/3)*d*c**(3/2))
def habs(m1,m2,d,f): # gravitational wave frequency
    return ampl(m1,m2,d)*f**(-7/6)

print(habs(12.4913,12.3884,10.8287*10**6*pc,1./cc))


1.1371737338193066e-14


In [ ]:
ampl(10*Msol,20*Msol,)

In [5]:
# here we define the SNR; 
# be careful that I have already put the factor for the sky average
# we use the GW only template.

def SNR(sky,fmin,fmax,m1,m2,d): # gravitational wave frequency
    return sqrt(4*integrate.quad(lambda x: (sky*habs(m1,m2,d, x/cc))**2/(Pn(x)*(cc**2)), fmin, fmax)[0])
#different noise here!

#print(SNR(0.01,fmax(30*Msol,10*Msol,0.01,Tobs),60*Msol,10*Msol,pi/2,400*10**6*pc),400*10**6*pc)


In [6]:
SNR(skyav*1/2,0.017058,0.4,23.5*Msol,22.57*Msol,40.45*10**6*pc)

45.42816185452258

In [7]:
# accretion phase
def faseacc(f,fm,m1,m2,xi,fEdd): # gravitational wave frequency
    return (25*factor*fEdd*(f*fm*Mc(m1,m2))**(2/3)*(13*f**(8/3)*fm**(5/3)*nu(m1,m2) + 12*f**(13/3)*(nu(m1,m2)- 2*xi)+fm**(13/3)*(-25*nu(m1,m2) + 24*xi)))/(159744*f**5*fm**5*pi**(13/3)*Mc(m1,m2)**4*nu(m1,m2))


In [7]:
# sky location functions
'''
def Aplus(iota,psi):
    return -(1.+cos(iota)**2.)*cos(2.*psi)-2.*cos(iota)*sin(2.*psi)
def Across(iota,psi):
    return (1.+cos(iota)**2.)*sin(2.*psi)-2.*cos(iota)*cos(2.*psi)
def Fplus(theta,phi):
    return 1./2.*(1.+cos(theta)**2.)*cos(2.*phi)
def Fcross(theta,phi):
    return cos(theta)*sin(2.*phi)

def factorsky(iota,psi,theta,phi): # absolute value
    return (Fplus(theta,phi)*Aplus(iota,psi))**2.+(Fcross(theta,phi)*Across(iota,psi))**2

def randomsky():
    #iota=random.uniform(0,pi)
    cosiota = random.uniform(-1.,1.)
    psi = random.uniform(0,2*pi)
    costheta=random.uniform(-1.,1.)
    phi=random.uniform(0,2*pi)
    return arccos(cosiota),psi,arccos(costheta),phi,factorsky(arccos(cosiota),psi,arccos(costheta),phi)
'''

In [9]:
# frequency functions 

def fMIN2(fmax0,m1,m2,Tobs): # gravitational wave frequency
    return 1/(1/fmax0**(8/3)+256/5*(Mc(m1,m2)/cc)**(5/3)*pi**(8/3)*(Tobs))**(3/8)

def fmax(m1,m2,fmin,Tobs): # gravitational wave frequency
    return fmin*((5*cc)/(5*cc-256*fmin**(8/3)*pi**(8/3)*Tobs*Mc(m1,m2)*(Mc(m1,m2)/cc)**(2/3)))**(3/8)


# gives fmax, given fmin and Tobs. If fmax is above 10 Hz, gives 10 Hz instead. frequency is always in Hz

def getfmax(m1,m2,fmin,Tobs): # gravitational wave frequency
    if fMIN2(1,m1,m2,Tobs)>fmin:
        return fmax(m1,m2,fmin,Tobs)
    else: 
        return 1
    
def Tmerger(m1,m2,fmin): # gravitational wave frequency
    return 5. * (m1 + m2)**(1./3.) * cc**(5./3.)/ (256. * fmin**(8./3.) * m1 * m2 * pi**(8./3.))

getfmax(29*Msol,36*Msol,0.01,10*year)
Tmerger(29*Msol,36*Msol,0.01)/year


17.009058428834408

In [17]:
Tmerger(60,80,0.01)/year

9.149848434605692

In [8]:
#here we define the mass models, already normalized

# mass model parameters
factorA = 14.188328379  ## laura
factorB = 51.15
factorS = 7.4307563728937

R0A = 10**(-9)*64.9/year; # Mpc^-3 s^-1
R0B = 10**(-9)*53.2/year;
R0S = 10**(-9)*32/year; # Log Flat distribution of Sesana's ("S") work

def modelloA(m1,m2): # 1/ km^2
    if (5.001 * Msol <= m2 <= m1 <= 208*Msol/5):
        return ((m1)**(-2/5)/(m1-5*Msol))/factorA
    return 0

def modelloB(m1,m2):
    if (7.9*Msol <= m2 <= m1 <= 42*Msol):
        return (7.7*(m1**(-1.6)*(m2/m1)**6.7)/(m1**(-8.3)*(m1**7.7-(7.9*Msol)**7.7)))/factorB
    return 0

def modelloS(m1,m2):
    if (m1 >= 5*Msol and m2 >= 5*Msol and m1 + m2 <= 100*Msol):
        return 1/(m1*m2*factorS)
    return 0
#here we define the cosmology and the different factors in the game

H0 = 67.9 # km /s / Mpc # changed to PLANCK 2015
Omegam = 0.306
OmegaL = 0.694
Omegar = 0.

def distL(z):
    return (1+z) * integrate.quad(invE,0,z)[0] *cc / H0 *10**6*pc # km

def Ez(z):
    return sqrt(Omegar*(1+z)**4+Omegam*(1+z)**3+OmegaL)

def invE(z):
    return 1. / Ez(z)

def primopezzo(zp):
    return 1/Ez(zp)*(integrate.quad(invE,0,zp)[0])**2

def dVdz(z):
    return 4*pi*(cc/H0)**3*primopezzo(z) # Mpc**3

def dtdf(f,m1,m2):  # Hz, km, km, gravitational wave frequency
    return 5/(96*pi**(8/3))*(1./cc*(m1 + m2)*(m1*m2/(m1 + m2)**2)**(3/5))**(-5/
  3)*f**(-11/3)  # s^2, assuming f in Hz and m in km

def solof(f):
    return f**(-11/3)

# here is our final distribution

def distribution(model,m1,m2,f,z):  #km, km, Hz
    if model == "A" :
        return R0A*modelloA(m1,m2)*dVdz(z)*dtdf(f,m1,m2) # units: Mpc^-3 s^-1 km^-2 Mpc**3 s^2 = s /km^2, ok!
    if model == "B" :
        return R0B*modelloB(m1,m2)*dVdz(z)*dtdf(f,m1,m2)
    elif model == "S":
        return R0S*modelloS(m1,m2)*dVdz(z)*dtdf(f,m1,m2)
    else :
        print("the input must be the string A or B or S for mass model A or B or S")



In [9]:
# mass dependent integrand

def massonlyA(m1,m2):
    return ((m1 + m2)*(m1*m2/(m1 + m2)**2)**(3/5))**(-5/3)*modelloA(m1,m2)

def massonlyB(m1,m2):
    return ((m1 + m2)*(m1*m2/(m1 + m2)**2)**(3/5))**(-5/3)*modelloB(m1,m2)

def massonlyS(m1,m2):
    return ((m1 + m2)*(m1*m2/(m1 + m2)**2)**(3/5))**(-5/3)*modelloS(m1,m2)
        
# the integrand is separable, so we integrate separately the z, f and m1, m2 dependences

def binAB(model,inz,finz,inf,finf,inm1,finm1,inm2,finm2):
    if model == "A" :
        return (integrate.quad(dVdz,inz,finz)[0]) * (integrate.quad(solof,inf,finf)[0]) * 5.*c**5./(96.*pi**(8./3.))*G**(-5./3.)*cc**(5./3.) * R0A * integrate.nquad(massonlyA,[[inm1,finm1],[inm2,finm2]])[0]
    elif model == "B" :
        return (integrate.quad(dVdz,inz,finz)[0]) * (integrate.quad(solof,inf,finf)[0])*5.*c**5./(96.*pi**(8./3.))*G**(-5./3.)*cc**(5./3.) * R0B * integrate.nquad(massonlyB,[[inm1,finm1],[inm2,finm2]])[0]
    elif model == "S":
        return (integrate.quad(dVdz,inz,finz)[0]) * (integrate.quad(solof,inf,finf)[0])*5.*c**5./(96.*pi**(8./3.))*G**(-5./3.)*cc**(5./3.) * R0S * integrate.nquad(massonlyS,[[inm1,finm1],[inm2,finm2]])[0]
    else :
        print("the input must be the string A or B or S for mass model A or B or S")


In [11]:
# rejection sampling function. 

def rejectionsampling(model,minz, maxz, minf, maxf, minm1, maxm1, minm2, maxm2):
    
    def mdist(vars):
        m1,m2,f,z = vars
        if minz <= z < maxz and minf <= f < maxf and minm1 <= m1 < maxm1 and minm2 < m2 < maxm2 : 
            if model == "A" :
                return - modelloA(m1,m2)*dVdz(z)*R0A*dtdf(f,m1,m2) # minus sign to use the optimize.fmin function
            if model == "B" :
                return - modelloB(m1,m2)*dVdz(z)*R0B*dtdf(f,m1,m2)
            if model == "S":
                return - modelloS(m1,m2)*dVdz(z)*R0S*dtdf(f,m1,m2)
            else :
                print("the input must be the string A or B or S for mass model A or B or S")
        else:
            return 0
            
    initial_guess = [minm1+.1, minm2, minf, minz]
    themax = scipy.optimize.fmin(mdist, initial_guess, ftol=0.000001, maxiter=1000, disp=False) # m1, m2, f, z
    themaxvalue = distribution(model,themax[0], themax[1], themax[2], themax[3]) 
    
    while True: 
        m1test = random.uniform(minm1, maxm1) 
        m2test = random.uniform(minm2, maxm2)
        ftest = random.uniform(minf, maxf)
        ztest = random.uniform(minz, maxz)
        y = random.uniform(0.,1.)*themaxvalue
        if(y < distribution(model,m1test, m2test, ftest, ztest)):
            break
    return [m1test, m2test, ftest, ztest]

In [12]:

def makemock(model): # Running over every bin. Computes the <N> events in a bin (integrating the distribution)
                # and defines a Poisson distribution with average <N>. Events are then picked
                # using the rejection sampling function, 
                # with a cut off to the number of events in a bin to avoid long running times.
    if model != "A" and model != "B" and model != "S":
        print("the input must be the string A or B or S for mass model A or B or S")
    for i in range(Nm):
        for j in range(Nm):
            for k in range(Nf):
                for n in range(Nz):
                    #if (i+j+k+n)%3:
                    #    print(i,j,k,n)
                    
                    minz = (zi+n*stepz)
                    maxz = (zi+(n+1)*stepz)
                    minf = (10**(fi+k*stepf))
                    maxf = (10**(fi+(k+1)*stepf))
                    minm1 = (Mi+i*stepM)  
                    maxm1 = (Mi+(i+1)*stepM)
                    minm2 = (Mi+j*stepM) 
                    maxm2 = (Mi+(j+1)*stepM)
                    
                    avern = binAB(model,minz, maxz, minf, maxf, minm1, maxm1, minm2, maxm2)
                    
                    print(avern)
                    
                    if avern < 100:
                        b = random.uniform(0,1)
                    
                        def poiss(numero):
                            return poisson.pmf(numero,avern)
                    
                        nev = 0
                        while True: 
                            if sum([poiss(l) for l in range(nev)]) <= b < sum([poiss(l) for l in range(nev+1)]):
                                eventi.append(nev)
                                for l in range(nev):
                                    newpoints = rejectionsampling(model,minz, maxz, minf, maxf, minm1, maxm1, minm2, maxm2)
                                    mockdatasetM1.append(newpoints[0])
                                    mockdatasetM2.append(newpoints[1])
                                    mockdatasetf.append(newpoints[2])                               
                                    mockdatasetz.append(newpoints[3])
                                break
                            elif sum([poiss(l) for l in range(nev+1)]) > 0.99: #or nev >= 5000: 
                                break    
                            nev +=1
                        print(nev)
                        
                    else:
                        mu, sigma = avern, sqrt(avern)
                        nev = int(round(np.random.normal(mu, sigma, 1)[0]))
                        print("Gaussian")
                        print(nev)
                        for l in range(nev):
                            newpoints = rejectionsampling(model,minz, maxz, minf, maxf, minm1, maxm1, minm2, maxm2)
                            mockdatasetM1.append(newpoints[0])
                            mockdatasetM2.append(newpoints[1])
                            mockdatasetf.append(newpoints[2])                               
                            mockdatasetz.append(newpoints[3])
                    
    return 0                        


In [14]:
# events above threshold calculator. TLISA must be in seconds, LIGO is 0 or 1

def appendall(aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR,z,fmin,m1,m2,snr,i):
    aboveth_mockdatasetz.append(z)
    aboveth_mockdatasetf.append(fmin)
    aboveth_mockdatasetM1.append(m1)
    aboveth_mockdatasetM2.append(m2)
    aboveth_SNR.append(snr)
    return 0

def snrthreshold(mockdatasetz, mockdatasetf, mockdatasetM1, mockdatasetM2, TLISA, LIGO):
    if LIGO != 0 and LIGO !=1 :
        print("error: choose LIGO=0 if LISA only, choose LIGO=1 if LIGO will detect.")
    
    aboveth_mockdatasetz=[]
    aboveth_mockdatasetf=[]
    aboveth_mockdatasetM1=[]
    aboveth_mockdatasetM2=[]
    aboveth_SNR=[]

    for i in range(len(mockdatasetM1)):
        z = mockdatasetz[i]
        Tobs = TLISA 
        m1 = mockdatasetM1[i]*(1+z)
        m2 = mockdatasetM2[i]*(1+z)
        fmin = mockdatasetf[i]/(1+z)
        Tm = Tmerger(m1,m2,fmin) # in our frame
        snr = 1./2.*SNR(sqrt(skyav),fmin, getfmax(m1,m2,fmin,Tobs), m1, m2, distL(z)) 
        
        if LIGO == 0 and Tm <= 10. * year and snr >= 15 :  # min SNR for LISA only
            appendall(aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR,z,fmin,m1,m2,snr,i)               
        if LIGO == 0 and 10 * year < Tm <= 100 * year and snr >= 15. : # LIGO doesn't matter here anyways
            appendall(aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR,z,fmin,m1,m2,snr,i)               
        if LIGO == 0 and Tm > 100 * year and snr >= 10. :
            appendall(aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR,z,fmin,m1,m2,snr,i)               
        if LIGO == 1 and Tm < 10. * year and snr >= 9.5:
            appendall(aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR,z,fmin,m1,m2,snr,i)               

    return aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR

# Mock data 

In [ ]:
# when running makemock(model), event parameters are saved in mockdatasetz, etc 

# the mock data function.

# set Tobs and model !

model = "A"

Nf = 10 # frequency bin 
Nz = 10 # red-shift bin
Nm = 10 # mass bin

# ranges 
zi = 0.
zf = 0.515 #1.
fi = -2.4 # maybe we could start from -2.5? depending on the mass, they could be visible
ff = 1.
Mi = 5. * Msol # ok for model A and B
Mf = 42. * Msol # 100. * Msol or 50 Msol for model B. 42 Msol is good for A and B.


# steps 
deltaz = zf-zi
deltaf = ff - fi
stepz = deltaz/Nz
stepf = deltaf/Nf
deltaM = Mf - Mi
stepM = deltaM/Nm

eventi=[]
checkavern=[]
mockdatasetz=[]
mockdatasetf=[]
mockdatasetM1=[]
mockdatasetM2=[]

start =  time.perf_counter()
makemock(model)
end =  time.perf_counter()
(end-start)/60 # in min.  234 min for z=0.4 without Gaussian option. 13 min with Gaussian! z=1, 70 min.

/Users/andreacaputo/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


1299.9280301652227
Gaussian
1294
8720.799403488832
Gaussian
8720
22536.591718367068
Gaussian
22786
41671.90170009664
Gaussian
42000
65117.695011754986
Gaussian
65380


In [ ]:
formdata = zeros((len(mockdatasetz),9))
for i in range(len(mockdatasetz)):
    formdata[i,0] = distL(mockdatasetz[i]) # km
    formdata[i,1] = mockdatasetf[i]/(1+mockdatasetz[i])
    formdata[i,2] = getfmax(mockdatasetM1[i],mockdatasetM2[i],mockdatasetf[i]/(1+mockdatasetz[i]),Tobs)
    formdata[i,3] = mockdatasetM1[i]
    formdata[i,4] = mockdatasetM2[i]
    formdata[i,5] = mockdatasetz[i]
np.savetxt('massA_alldata1.txt', formdata, fmt="%s")

# Threshold, with LIGO, 10 years: 

In [26]:
# run this to find the mock data above threshold, and save into the following arrays
# -2.5 268
LIGO = 1
Tobs = 10 * year

'''
del aboveth_mockdatasetz
del aboveth_mockdatasetf
del aboveth_mockdatasetM1
del aboveth_mockdatasetM2
del aboveth_SNR
'''

aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR = snrthreshold(mockdatasetz, mockdatasetf, mockdatasetM1, mockdatasetM2, Tobs, LIGO)
print(len(aboveth_mockdatasetz)) #saves redshifted quantities


14


In [27]:
# to save to file

formdata = zeros((len(aboveth_mockdatasetz),9))
for i in range(len(aboveth_mockdatasetz)):
    formdata[i,0] = distL(aboveth_mockdatasetz[i]) # km
    formdata[i,1] = aboveth_mockdatasetf[i]
    formdata[i,2] = getfmax(aboveth_mockdatasetM1[i],aboveth_mockdatasetM2[i],aboveth_mockdatasetf[i],Tobs)
    formdata[i,3] = aboveth_mockdatasetM1[i]
    formdata[i,4] = aboveth_mockdatasetM2[i]
    formdata[i,5] = aboveth_SNR[i]
    formdata[i,6] = random.uniform(0.,1.) # spin1
    formdata[i,7] = random.uniform(0.,1.) # spin2
    formdata[i,8] = random.uniform(0.,2.*pi) # phase
np.savetxt('massB_zmax05_f24_aboveth10yearsLIGO_1.txt', formdata, fmt="%s")

# Threshold, with LIGO, 4 years: 

In [28]:
# run this to find the mock data above threshold, and save into the following arrays
# -2.5 268
LIGO = 1
Tobs = 4 * year

del aboveth_mockdatasetz
del aboveth_mockdatasetf
del aboveth_mockdatasetM1
del aboveth_mockdatasetM2
del aboveth_SNR

aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR = snrthreshold(mockdatasetz, mockdatasetf, mockdatasetM1, mockdatasetM2, Tobs, LIGO)
print(len(aboveth_mockdatasetz)) #saves redshifted quantities


6


In [29]:
# to save to file

formdata = zeros((len(aboveth_mockdatasetz),9))
for i in range(len(aboveth_mockdatasetz)):
    formdata[i,0] = distL(aboveth_mockdatasetz[i]) # km
    formdata[i,1] = aboveth_mockdatasetf[i]
    formdata[i,2] = getfmax(aboveth_mockdatasetM1[i],aboveth_mockdatasetM2[i],aboveth_mockdatasetf[i],Tobs)
    formdata[i,3] = aboveth_mockdatasetM1[i]
    formdata[i,4] = aboveth_mockdatasetM2[i]
    formdata[i,5] = aboveth_SNR[i]
    formdata[i,6] = random.uniform(0.,1.) # spin1
    formdata[i,7] = random.uniform(0.,1.) # spin2
    formdata[i,8] = random.uniform(0.,2.*pi) # phase
np.savetxt('massB_zmax05_f24_aboveth4yearsLIGO_1.txt', formdata, fmt="%s")

# Threshold, without LIGO, 10 years: 

In [31]:
# run this to find the mock data above threshold, and save into the following arrays
# -2.5 268
LIGO = 0
Tobs = 10 * year

del aboveth_mockdatasetz
del aboveth_mockdatasetf
del aboveth_mockdatasetM1
del aboveth_mockdatasetM2
del aboveth_SNR

aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR = snrthreshold(mockdatasetz, mockdatasetf, mockdatasetM1, mockdatasetM2, Tobs, LIGO)
print(len(aboveth_mockdatasetz)) #saves redshifted quantities


93


In [32]:
# to save to file

formdata = zeros((len(aboveth_mockdatasetz),9))
for i in range(len(aboveth_mockdatasetz)):
    formdata[i,0] = distL(aboveth_mockdatasetz[i]) # km
    formdata[i,1] = aboveth_mockdatasetf[i]
    formdata[i,2] = getfmax(aboveth_mockdatasetM1[i],aboveth_mockdatasetM2[i],aboveth_mockdatasetf[i],Tobs)
    formdata[i,3] = aboveth_mockdatasetM1[i]
    formdata[i,4] = aboveth_mockdatasetM2[i]
    formdata[i,5] = aboveth_SNR[i]
    formdata[i,6] = random.uniform(0.,1.) # spin1
    formdata[i,7] = random.uniform(0.,1.) # spin2
    formdata[i,8] = random.uniform(0.,2.*pi) # phase
np.savetxt('massB_zmax05_f24_aboveth10years_1.txt', formdata, fmt="%s")

# Threshold, without LIGO, 4 years: 

In [33]:
# run this to find the mock data above threshold, and save into the following arrays
# -2.5 268
LIGO = 0
Tobs = 4 * year

del aboveth_mockdatasetz
del aboveth_mockdatasetf
del aboveth_mockdatasetM1
del aboveth_mockdatasetM2
del aboveth_SNR

aboveth_mockdatasetz,aboveth_mockdatasetf,aboveth_mockdatasetM1,aboveth_mockdatasetM2,aboveth_SNR = snrthreshold(mockdatasetz, mockdatasetf, mockdatasetM1, mockdatasetM2, Tobs, LIGO)
print(len(aboveth_mockdatasetz)) #saves redshifted quantities


44


In [34]:
# to save to file

formdata = zeros((len(aboveth_mockdatasetz),9))
for i in range(len(aboveth_mockdatasetz)):
    formdata[i,0] = distL(aboveth_mockdatasetz[i]) # km
    formdata[i,1] = aboveth_mockdatasetf[i]
    formdata[i,2] = getfmax(aboveth_mockdatasetM1[i],aboveth_mockdatasetM2[i],aboveth_mockdatasetf[i],Tobs)
    formdata[i,3] = aboveth_mockdatasetM1[i]
    formdata[i,4] = aboveth_mockdatasetM2[i]
    formdata[i,5] = aboveth_SNR[i]
    formdata[i,6] = random.uniform(0.,1.) # spin1
    formdata[i,7] = random.uniform(0.,1.) # spin2
    formdata[i,8] = random.uniform(0.,2.*pi) # phase
np.savetxt('massB_zmax05_f24_aboveth4years_1.txt', formdata, fmt="%s")